In [7]:
import random
from game import Game, Move, Player
from strategies.rl import Q_learing, CustomState,  get_coordinates

class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move


class MyPlayer(Player):
    def __init__(self) -> None:
        super().__init__()

    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move
    
class RLPlayer(Player):
    def __init__(self, steps, learning_rate, discount_factor) -> None:
        super().__init__()
        ql = Q_learing(steps, learning_rate, discount_factor)
        self.value_dictionary = ql.train()
        
    def make_move(self, game: Game) -> tuple[tuple[int, int], Move]:
        current_state = CustomState(get_coordinates(game.get_board()))
        list_action = sorted(self.value_dictionary[current_state], key=self.value_dictionary[current_state].get)
        '''
        for action in list_action:
            if action not in (current_state.x.union(current_state.o)):
                current_state.x.add(action)
                break
        '''
        print(list_action[0])
        from_pos_0,from_pos_1, move = list_action[0].split()
        from_pos=from_pos_0.split+from_pos_1
        print(from_pos)
        from_pos=tuple(map(int, from_pos.split(',')))
     
        return from_pos, move
    
g = Game()

player1 = RLPlayer(100, 0.1, 0.7)
player2 = RandomPlayer()



  0%|          | 0/100 [00:00<?, ?it/s]

state(x=[], o=[]):
  (2, 4) Move.LEFT: 0.0
  (4, 2) Move.TOP: 0.0
  (0, 4) Move.RIGHT: 0.0
  (4, 3) Move.LEFT: 0.0
  (0, 1) Move.BOTTOM: 0.0
  (3, 4) Move.LEFT: 0.0
  (0, 0) Move.BOTTOM: 0.0
  (0, 2) Move.BOTTOM: 0.0
  (2, 0) Move.BOTTOM: 0.0
  (4, 4) Move.LEFT: 0.0
  (3, 0) Move.BOTTOM: 0.0
  (2, 4) Move.TOP: 0.0
  (0, 2) Move.RIGHT: 0.0
  (1, 0) Move.LEFT: 0.0
  (2, 0) Move.RIGHT: 0.0
  (0, 3) Move.BOTTOM: 0.0
  (1, 4) Move.LEFT: 0.0
  (0, 3) Move.RIGHT: 0.0
  (1, 4) Move.RIGHT: 0.0
  (3, 4) Move.TOP: 0.0
  (0, 0) Move.RIGHT: 0.0
  (2, 0) Move.LEFT: 0.0
  (4, 1) Move.TOP: 0.0
  (0, 4) Move.TOP: 0.0
  (0, 3) Move.TOP: 0.0
  (3, 0) Move.RIGHT: 0.0
  (4, 4) Move.TOP: 0.0
  (4, 3) Move.TOP: 0.0
  (3, 4) Move.RIGHT: 0.0
  (3, 0) Move.LEFT: 0.0
  (1, 4) Move.TOP: 0.0
  (0, 2) Move.TOP: 0.0
  (4, 2) Move.BOTTOM: 0.0
  (4, 0) Move.BOTTOM: 0.0
  (4, 0) Move.LEFT: 0.0
  (4, 2) Move.LEFT: 0.0
  (0, 1) Move.RIGHT: 0.0
  (0, 1) Move.TOP: 0.0
state(x=[(0, 0)], o=[]):
  (2, 4) Move.LEFT: 0.0
  (3, 

In [8]:
g.print()
winner = g.play(player1, player2)
g.print()
print(f"Winner: Player {winner}")

[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]
[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]
(2, 4) Move.LEFT


ValueError: invalid literal for int() with base 10: '(2'

In [1]:
from strategies.utils import CustomGame
from strategies.utils import MinMaxPlayer,RandomPlayer
import random
from game import Game, Move, Player
from strategies.rl import Q_learing, CustomState,  get_coordinates
cnt=0
for i in range(100):
    game=CustomGame()
    player1=MinMaxPlayer()
    player2=RandomPlayer()
    winner=game.play(player1,player2)
    if winner==0:
        cnt+=1
    
    print(cnt)

1
1
1
2
2


KeyboardInterrupt: 